Importamos librerias

In [87]:
import joblib
import os

import pandas as pd
import numpy as np

import sys
sys.path.append('../../')
#from utils_yose import build_preprocessor

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.5f' % x)

Cargar archivos necesarios

In [ ]:
model_dir = "../../models/pipemodelFINAL"
data_dir = "../../../data/housing_data"

final_pipe_lasso = joblib.load(os.path.join(model_dir, "final_pipe_lasso.joblib"))
final_pipe_lgbm = joblib.load(os.path.join(model_dir, "final_pipe_lgbm.joblib"))
final_pipe_cat = joblib.load(os.path.join(model_dir, "final_pipe_catboost.joblib"))
final_pipe_ridge = joblib.load(os.path.join(model_dir, "final_pipe_ridge.joblib"))

opt_weights = pd.read_json(os.path.join(model_dir, "meta.json"))['ensemble_weights'].dropna()

def predict_3pipes(X, weights=opt_weights):
    return np.expm1(final_pipe_lasso.predict(X) * weights["lasso"] + final_pipe_ridge.predict(X) * weights["ridge"] + final_pipe_cat.predict(X) * weights["catboost"] + final_pipe_lgbm.predict(X) * weights["lgbm"])

df_test = pd.read_csv(os.path.join(data_dir, "train.csv"))

process input data

In [89]:
from sklearn.metrics import mean_squared_error, r2_score

X = df_test.drop(["SalePrice", "Id"], axis=1)
y = df_test["SalePrice"].astype(float)

y_pred = predict_3pipes(X)

rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)

print(f"RMSE: {rmse:.5f}")
print(f"R2: {r2:.5f}")

RMSE: 7374.15071
R2: 0.99138
